In [25]:
#import
import cv2
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.transforms.functional as F
from torchvision import models
from PIL import Image
import pandas as pd

from AP3D import API3D

import spatial_transforms as ST
from ResNet import AP3DResNet50

In [26]:
## 4 Frames 
img1 = Image.open("000003.jpg")
img2 = Image.open("000004.jpg")
img3 = Image.open("000005.jpg")
img4 = Image.open("000006.jpg")

In [27]:
open("gt.txt", "r")
Koordinaten = open("gt.txt", "r")

data = pd.read_csv('gt.txt', header = None)
data.columns = ['Frame', 'ID', 'x', 'y', 'w', 'h', 'Dies', 'Das', 'Visability']

data.drop(data[data.ID != 2].index, inplace=True)
data = data.loc[data['Frame'].isin(['3','4','5','6'])]
data.drop(['Frame', 'ID', 'Dies', 'Das', 'Visability'], axis=1, inplace=True)
print(data)

x1 = data.at[1052, 'x']
y1 = data.at[1052, 'y']
w1 = data.at[1052, 'w']
h1 = data.at[1052, 'h']

x2 = data.at[1053, 'x']
y2 = data.at[1053, 'y']
w2 = data.at[1053, 'w']
h2 = data.at[1053, 'h']

x3 = data.at[1054, 'x']
y3 = data.at[1054, 'y']
w3 = data.at[1054, 'w']
h3 = data.at[1054, 'h']

x4 = data.at[1055, 'x']
y4 = data.at[1055, 'y']
w4 = data.at[1055, 'w']
h4 = data.at[1055, 'h']

        x    y   w    h
1052  372  407  80  239
1053  372  406  81  239
1054  373  405  81  239
1055  373  404  82  238


In [28]:
## hardcode 4 bounding boxes
bbox1 = img1.crop((x1, y1, x1+w1, y1+h1))
bbox2 = img2.crop((x2, y2, x2+w2, y2+h2))
bbox3 = img3.crop((x3, y3, x3+w3, y3+h3))
bbox4 = img4.crop((x4, y4, x4+w4, y4+h4))

bbox1.save('bbox1.png')
bbox2.save('bbox2.png')
bbox3.save('bbox3.png')
bbox4.save('bbox4.png')

In [32]:
##brauche ich für das 3DCNN wieder die normalization und resize???
##transform = transforms.Compose([
##    transforms.Resize(256),
##    transforms.ToTensor(),
##    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
##])

##transform = ST.Compose([
##    ST.Scale((256, 128), interpolation=3),
##    ST.ToTensor(),
##    ST.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
##])

transform = transforms.Compose([
    transforms.Scale((256, 128), interpolation=3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

bbox1 = transform (bbox1)
bbox2 = transform (bbox2)
bbox3 = transform (bbox3)
bbox4 = transform (bbox4)

TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>

In [33]:
##print(bbox1.size())
##print(bbox2.size())
##print(bbox3.size())
##print(bbox4.size())

##bbox1_size = bbox1.unsqueeze(0)                        ##nimmt das 3DCNN 3 oder 4 dimensionale inputs
##bbox2_size = bbox2.unsqueeze(0)
##bbox3_size = bbox3.unsqueeze(0)
##bbox4_size = bbox4.unsqueeze(0)

## 4 bboxen zu einem Torch.Tensor machen
#bboxtensor = []
##bboxtensor.append(bbox1)
##bboxtensor.append(bbox2)
##bboxtensor.append(bbox3)
##bboxtensor.append(bbox4)

##bboxtensor = torch.Tensor(bboxtensor)                   ##only one element tensors can be converted to Python scalars


#bboxtensor = torch.stack(bbox1_size + bbox2_size + bbox3_size + bbox4_size) 
                                                          ##The size of tensor a (80) must match the size of tensor b (81)

    
bboxtensor = torch.Tensor(bbox1 + bbox2 + bbox3 + bbox4)
bboxtensor.size()

torch.Size([3, 256, 128])

In [34]:
##def AP3DResNet50(num_classes, **kwargs):
##    c3d_idx = [[],[0, 2],[0, 2, 4],[]]
##    nl_idx = [[],[],[],[]]

##    return ResNet503D(num_classes, AP3D.APP3DC, c3d_idx, nl_idx, **kwargs)
CNN3D = AP3DResNet50(1261)

## loading the model                                         ## ist conv2d (in_channels) mein input Vektor?
##(CNN3D = API3D(conv2d, time_dim=3, time_stride=1, temperature=4, contrastive_att=True))
## loading in weights
CNN3D.load_state_dict(torch.load('/home/ga27qef/thesis/resnet3d_mars.pth.tar'))

NameError: name 'AP3D' is not defined

In [35]:
features = CNN3D(bboxtensor)

NameError: name 'CNN3D' is not defined